# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [2]:
# your code here
import pandas as pd

In [4]:
df = pd.read_csv("tic-tac-toe.csv")

In [ ]:
df.head()

In [12]:
df = df.map(lambda x: 1 if x == 'x' else (0 if x == 'o' else -1))

In [15]:
X = df.iloc[:, :-1]  
y = df.iloc[:, -1]

In [17]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [20]:
# your code here

In [ ]:
!pip install tensorflow

In [ ]:
!pip install keras

In [26]:
from sklearn.model_selection import train_test_split

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()

In [45]:
# Add layers to the model
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(3, activation='softmax'))


In [47]:
# Remap labels (adjust the mapping based on your dataset)
y = y.map({-1: 0, 0: 1, 1: 2})

In [ ]:
print(y.unique())

In [50]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)


In [51]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)


In [ ]:
print(y_train.unique())

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [ ]:
# your code here
predictions = model.predict(X_test)


In [ ]:
# Get the class with the highest probability
predicted_classes = predictions.argmax(axis=1)
print(predicted_classes)


In [ ]:
# Apply threshold to classify as 0 or 1
predicted_classes = (predictions > 0.5).astype(int)
print(predicted_classes)


In [ ]:
model.save('tic-tac-toe.keras')
print("Model saved as 'tic-tac-toe.keras'.")

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [67]:
# your code here
from tensorflow.keras.optimizers import Adam


learning_rate = 0.0005 
optimizer = Adam(learning_rate=learning_rate)


model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Fit the model with more epochs
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)


**Which approach(es) did you find helpful to improve your model performance?**

In [1]:
# your answer here

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

# Generate a synthetic spiral dataset (or use another dataset like moons)
def generate_spiral_data(samples, classes):
    X = np.zeros((samples * classes, 2))
    y = np.zeros(samples * classes, dtype="uint8")
    for class_number in range(classes):
        ix = range(samples * class_number, samples * (class_number + 1))
        r = np.linspace(0.0, 1, samples)  # radius
        t = np.linspace(class_number * 4, (class_number + 1) * 4, samples) + np.random.randn(samples) * 0.2  # theta
        X[ix] = np.c_[r * np.sin(t), r * np.cos(t)]
        y[ix] = class_number
    return X, y

# Create spiral dataset
X, y = generate_spiral_data(100, 3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Visualize dataset
plt.scatter(X[:, 0], X[:, 1], c=y, cmap="brg")
plt.show()

# Build the model
model = models.Sequential()
model.add(layers.Dense(64, activation="relu", input_shape=(2,)))  # Input layer
model.add(layers.Dense(64, activation="relu"))  # Hidden layer
model.add(layers.Dense(3, activation="softmax"))  # Output layer (3 classes)

# Compile the model with hyperparameters
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),  # Adjust learning rate
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Train the model
history = model.fit(X_train, y_train, epochs=500, batch_size=16, validation_split=0.2, verbose=0)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

# Plot predictions
y_pred = np.argmax(model.predict(X_test), axis=1)
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_pred, cmap="brg", alpha=0.5)
plt.title("Predicted Classes")
plt.show()
